In [ ]:
!pip install -r requirements.txt 

In [10]:
import requests
import gspread
from io import BytesIO
from PIL import Image
from google.oauth2.service_account import Credentials
import pandas as pd

In [22]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('PATH_YOUR_CREDENTIALS/auth_cred.json', scopes=scope)
sheet_id = 'WRITE_YOUR_SHEET_ID' # example: https://docs.google.com/spreadsheets/d/1Qb1ybwIQ3TjOkk4PfHQ_1N87eJ0TLfqOujxc/edit#gid=0 

Examle:
Link sheet: https://docs.google.com/spreadsheets/d/1Qb1ybwIQ3TjOkk4PfHQ_1N87eJ0TLfqOujxc/edit#gid=0 
sheet_id: 1Qb1ybwIQ3TjOkk4PfHQ_1N87eJ0TLfqOujxc


In [15]:
client = gspread.authorize(creds) 
sheet_place = client.open_by_key(sheet_id)

worksheet = sheet_place.get_worksheet(0)
data_sheet = worksheet.get_all_values()
df_sheet = pd.DataFrame(data_sheet[1:], columns=data[0])

In [16]:
df_sheet # look on sheet data

,image_url,SIZE
0,https://data.sanitino.eu/PRODUCT-33916/8607663...,575x860
1,https://data.sanitino.eu/PRODUCT-62434/f7aa3c2...,
2,https://data.sanitino.eu/PRODUCT-33915/8607663...,
3,https://data.sanitino.eu/PRODUCT-62426/506d67b...,
4,https://data.sanitino.eu/PRODUCT-33893/ccd99b4...,
...,...,...
129,https://data.sanitino.eu/PRODUCT-29341/071b923...,
130,https://data.sanitino.eu/PRODUCT-29337/f72de61...,
131,https://data.sanitino.eu/PRODUCT-29340/071b923...,
132,https://data.sanitino.eu/PRODUCT-62461/b6a28a3...,


In [18]:
def image_parametrs(image_url: str):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(image_url, headers=headers)
    if response.status_code != 200:
        return f"status link: {response.status_code}"
    image = Image.open(BytesIO(response.content))
    width, height = image.size
    return f"{width}x{height}" 

In [19]:
df_sheet['SIZE'] = df_sheet['image_url'].apply(image_parametrs)
df_sheet

,image_url,SIZE
0,https://data.sanitino.eu/PRODUCT-33916/8607663...,1080x1614
1,https://data.sanitino.eu/PRODUCT-62434/f7aa3c2...,1080x1080
2,https://data.sanitino.eu/PRODUCT-33915/8607663...,1080x1614
3,https://data.sanitino.eu/PRODUCT-62426/506d67b...,1080x1080
4,https://data.sanitino.eu/PRODUCT-33893/ccd99b4...,1080x1614
...,...,...
129,https://data.sanitino.eu/PRODUCT-29341/071b923...,1093x1080
130,https://data.sanitino.eu/PRODUCT-29337/f72de61...,1093x1080
131,https://data.sanitino.eu/PRODUCT-29340/071b923...,1093x1080
132,https://data.sanitino.eu/PRODUCT-62461/b6a28a3...,1422x1080


In [21]:
worksheet.update([df_sheet.columns.values.tolist()] + df_sheet.values.tolist())

{'spreadsheetId': '1Qb1ybwIQ3TjOkk4PfHApoQ_1N87eJ0TLfqOujxchagE',
 'updatedRange': 'feed!A1:B135',
 'updatedRows': 135,
 'updatedColumns': 2,
 'updatedCells': 270}